In [1]:
import os
from groq import Groq
from dotenv import load_dotenv
load_dotenv()
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.llms import OllamaLLM
from langchain_groq import ChatGroq
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
import streamlit as st

In [2]:
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [3]:
# docs = PyPDFLoader("/home/skynet/Documents/Gen_ai/GeetaGpt/data/bhagvatgeeta.pdf").load()

In [4]:
# splitter = RecursiveCharacterTextSplitter(chunk_size=8100,chunk_overlap=2000)

In [5]:
# corpus = splitter.split_documents(data)

In [6]:
model = ChatGroq(model="gemma2-9b-it",max_tokens=4050)
llm = OllamaLLM(model="gemma2:9b")
embeddings = OllamaEmbeddings(model="gemma2:9b")

In [7]:
# db = FAISS.from_documents(corpus, embeddings)

In [8]:
# db.save_local("bhagvatgeeta_faiss")

In [9]:
db = FAISS.load_local("bhagvatgeeta_faiss",embeddings=embeddings,allow_dangerous_deserialization=True)

In [10]:
# query = "What is the meaning of life?"
# results = db.similarity_search(query, k=1)

In [11]:
retriever = db.as_retriever()
# prompt = hub.pull('rlm/rag-prompt')

In [54]:
prompt_template = """
Prompt:

    Task: Answer questions based on the Bhagavad Gita teachings.

    Instructions:

        Retrieve relevant information: Search the Bhagavad Gita for passages that are most relevant to the query.
        Summarize key points: Extract the main ideas and concepts from the retrieved passages.
        Generate a comprehensive response: Combine the summarized information with the query to create a coherent and informative answer.
        Mention references: Clearly indicate the chapter and verse numbers from which the information is extracted.
        Mention Sholkas: Clearly include relevant shlokas from the Gita to support your response.
        Provide accurate information: Ensure that the information provided is accurate and consistent with the teachings of the Gita.
        Translate: Provide translations of the shlokas in Hindi to help explain their meaning.
        Stay focused: Keep your response focused on the query and avoid irrelevant information.
        Be concise: Provide a clear and concise response that addresses the query effectively.


    Guidelines:

        Stay true to the text: Ensure that your responses are consistent with the teachings of the Bhagavad Gita.
        Provide context: If applicable, provide additional context from the Gita to support your answer.
        Be respectful: Maintain a respectful and reverent tone when answering questions about the Bhagavad Gita.
        Be concise: Provide clear and concise responses that address the query directly.
        Stay focused: Ensure that your response is directly related to the Bhagavad Gita and avoids unrelated and out of context topics.
        Answering Style: Start with "Jai Shri Krishna" and end with "Jai Shri Krishna" in your answer.
        Out of scope: Avoid providing personal opinions or interpretations that are not supported by the text of the Gita.
        Out of Context: Avoid providing information that is not directly related to the query or the teachings of the Gita.
        Inaccurate Information: Avoid providing information that is inaccurate or inconsistent with the teachings of the Gita.
        Not Sure: If you don't know the answer: Say "Jai Shri Krishna! I'm not sure, would you like me to look it up?"

        
question:
{question}

"""

In [55]:
new_prompt = ChatPromptTemplate([
    prompt_template
])

In [56]:
rag_chain = (
    {"question": RunnablePassthrough()}
    | new_prompt
    | model
    | StrOutputParser()
)

In [57]:
rag_chain.invoke(
    "What is the significance of the Kṣetra-Kṣetreśvara concept in the Bhagavad Gita?"
)

'Jai Shri Krishna!\n\nThe concept of Kṣetra-Kṣetreśvara is of paramount significance in the Bhagavad Gita. It establishes the relationship between the individual soul (Kṣetra) and the Supreme Soul (Kṣetreśvara). \n\nThe Bhagavad Gita explains:\n\n**"You are the field, and I am the Lord of the field." (Chapter 13, Verse 1)  \n"तत् क्षेत्रं तेन तत्त्वतः कृषि  तेषु तत्क्षेत्रेश्वरं मम"**\n\nHere, the "field" (Kṣetra) represents the body, mind, and intellect, which are the instruments of the individual soul. The "Lord of the field" (Kṣetreśvara) is Krishna, representing the Supreme Soul who resides within every being. \n\nThis concept emphasizes:\n\n* **The Supreme Soul\'s omnipresence:** Krishna is not separate from the individual soul but resides within it.\n* **The individual\'s role as a field:** The individual is a vessel through which the Supreme Soul manifests.\n* **The importance of surrendering to the Supreme:** Recognizing Krishna as the Kṣetreśvara leads to true knowledge and li

In [ ]:
st.title("Bhagavad Gita Chatbot")
st.write(
    "This is a chatbot that can answer questions based on the teachings of the Bhagavad Gita."
)
st.write("Please enter your question below and click 'Ask' to get an answer.")
st.write(
    "You can ask questions about the teachings, philosophy, and concepts of the Bhagavad Gita."
)
st.write(
    "For example, you can ask about the meaning of life, the purpose of human existence, the nature of the soul, etc."
)
st.write(
    "Please note that this chatbot is designed to provide information based on the text of the Bhagavad Gita and may not be able to answer all questions."
)
# taking input from user
question = st.text_input("Enter your question here:")
if st.button("Ask"):
    answer = rag_chain.invoke(question)
    st.write(answer)

In [18]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [36]:
# The purpose of create stuff documents chain is to combine the documents into a single document to give the context to the model
document_chain = create_stuff_documents_chain(model,prompt=new_prompt)

In [42]:
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nPrompt:\n\n    Task: Answer questions based on the Bhagavad Gita teachings.\n\n    Instructions:\n\n        Retrieve relevant information: Search the Bhagavad Gita for passages that are most relevant to the query.\n        Summarize key points: Extract the main ideas and concepts from the retrieved passages.\n        Generate a comprehensive response: Combine the summarized information with the query to create a coherent and informative answer.\n        Mention references: Clearly indicate the chapter and verse numbers from which the information is extracted.\n        Mention Sholkas: Clearly include relevant shlokas from the Gita to support your response.\n        Provide accurate informatio

In [38]:
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [39]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fb374bfc8e0>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nPrompt:\n\n    Task: Answer questions based on the Bhagavad Gita teachings.\n\n    Instructions:\n\n        Retrieve relevant information: Search the Bhagavad Gita for passages that are most relevant to the query.\n        Summarize key points: Extract the main ideas and concepts from the retrieved passages.

In [41]:
response = retrieval_chain.invoke({"input":"What is the meaning of life?"})
response['answer']

'Jai Shri Krishna \n\nThe provided text discusses the nature of the soul and its relationship to the material world. \n\nIt emphasizes the eternal and unchanging nature of the soul, distinct from the changing body. Just as a tree sheds its leaves but remains, the soul transcends the limitations of the physical form. The soul is described as  "jīva-bhūtaḥ sanātanaḥ" (eternal living entity) and is full of consciousness. However, this consciousness is often obscured by material attachments.\n\nThe text also highlights the importance of spiritual guidance, stating that true knowledge of the soul and its liberation can be attained through the disciplic succession. It refers to  "evaṁ paramparā-prāptam imaṁ rājarṣayo viduḥ" (This Bhagavad-gītā As It Is is received through this disciplic succession), emphasizing the need to learn from qualified teachers.\n\nThe passage concludes by explaining that  "tad viddhi praṇipātena paripraśnena sevayā upadekṣyanti te jñānam jñāninas tattva-darśinaḥ" (T

In [25]:
document_chain.invoke(
    {
        "context": retriever.invoke(
            "Explain the Yoga of Knowledge (Jñāna Yoga) and Yoga of Action (Karma Yoga) as presented in the Gita."
        ),
        "question": "Explain the Yoga of Knowledge (Jñāna Yoga) and Yoga of Action (Karma Yoga) as presented in the Gita.",
    }
)

'Jai Shri Krishna!\n\nThe Bhagavad Gita presents two primary paths to liberation: Jñana Yoga (Yoga of Knowledge) and Karma Yoga (Yoga of Action). \n\n**Jñana Yoga (Yoga of Knowledge)**\n\nJñana Yoga emphasizes the pursuit of knowledge and wisdom about the true nature of reality, culminating in the realization of the Self (Atman) as identical to the Supreme Brahman. \n\n* **Key Principles:**\n    * **Discrimination (Viveka):** Distinguishing between the permanent (Atman) and the impermanent (material world).\n    * **Self-Study (Swadhyaya):** Deep introspection and study of scriptures to understand the Self and Brahman.\n    * **Meditation (Dhyana):** Concentrating the mind on the object of knowledge, ultimately leading to a state of pure consciousness.\n    * **Surrender (Sharanagati):**  Complete surrender to the Divine will, trusting in the guidance of a spiritual master.\n\n* **Bhagavad Gita Verses:**\n    * **Chapter 4, Verse 34:**  "yoga-siddhiṁ  śrimad-bhagavān  u-vacāḥ  \n    * 

In [89]:
def loading_models():
    return ChatGroq(model="gemma2-9b-it"), OllamaEmbeddings(model="gemma2:9b")

In [92]:
model, embeddings = loading_models()[0], loading_models()[1]

In [94]:
embeddings

OllamaEmbeddings(model='gemma2:9b', base_url=None, client_kwargs={}, _client=<ollama._client.Client object at 0x7f742c32cfd0>, _async_client=<ollama._client.AsyncClient object at 0x7f742c32cfa0>)

AIMessage(content="As a large language model, I don't have personal beliefs or experiences to draw on, so I can't tell you the meaning of life. That's a deeply philosophical question that humans have been pondering for centuries.\n\nThere isn't one answer that will satisfy everyone, as the meaning of life is often unique to each individual.\n\nHowever, I can offer you some perspectives:\n\n* **Finding Purpose:** Many people find meaning in pursuing goals, contributing to society, or making a difference in the world.\n* **Connecting with Others:** Relationships with loved ones, friends, and community can provide a sense of belonging and purpose.\n* **Experiencing the World:** Some find meaning in exploring new places, learning new things, and appreciating the beauty around them.\n* **Personal Growth:** The journey of self-discovery and personal growth can be a meaningful pursuit in itself.\n* **Creating Something:** Whether it's art, music, writing, or simply a kind act, creating someth